In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import pickle

import neurokit2 as nk

import seaborn as sns

%matplotlib qt
mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('ggplot')
plt.style.use('seaborn-v0_8-whitegrid')

# sns.set_context("talk")
sns.set_palette("Set1")

In [2]:
import pyreadr
from collections import Counter
from functions.bursts import *
from sleep_diary import diary_SPT

# HR response

## Sleep vs Wake vs SPT

In [3]:
import matplotlib.ticker as mtick

In [4]:
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]
bursts_HR = pd.read_pickle("/Volumes/Untitled/rehab/data/bursts_HR_ACC_final.pkl")
bursts_HR["ACC_response"] = bursts_HR["ACC_response"].apply(lambda x: np.array(x))
print(bursts_HR.shape)

(834, 15)


In [5]:
bursts_HR["Duration"] = bursts_HR["End"] - bursts_HR["Start"]
bursts_HR["Duration"] = bursts_HR["Duration"].apply(lambda x: x.total_seconds())
bursts_HR["n_limbs"] = bursts_HR["Limbs"].apply(lambda x: len(x))
bursts_HR["Category"] = "placeholder"
bursts_HR["Laterality"] = "Bilateral"
bursts_HR.loc[bursts_HR["n_limbs"] == 5, "Category"] = "Full Body"
bursts_HR.loc[(bursts_HR["Limbs"] == {"LL", "RL"}) | (bursts_HR["Limbs"] == {"LL", "T"}) | (bursts_HR["Limbs"] == {"RL", "T"}) | (bursts_HR["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_HR.loc[(bursts_HR["Limbs"] == {"LW", "RW"}) | (bursts_HR["Limbs"] == {"LW", "T"}) | (bursts_HR["Limbs"] == {"RW", "T"}) | (bursts_HR["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_HR.loc[(bursts_HR["n_limbs"] >= 2) & (bursts_HR["n_limbs"] != 5) & (bursts_HR["Limbs"] != {"LL", "RL"}) & (bursts_HR["Limbs"] != {"LL", "RL", "T"}) & (bursts_HR["Limbs"] != {"LW", "RW"}) & (bursts_HR["Limbs"] != {"LW", "RW", "T"}) & (bursts_HR["Limbs"] != {"LL", "T"}) & (bursts_HR["Limbs"] != {"RL", "T"}) & (bursts_HR["Limbs"] != {"LW", "T"}) & (bursts_HR["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_HR.loc[bursts_HR["n_limbs"] == 1, "Category"] = "Focal"
bursts_HR.loc[(bursts_HR["Limbs"] == {"LL", "LW"}) | (bursts_HR["Limbs"] == {"RL", "RW"}) | (bursts_HR["Limbs"] == {"LL", "LW", "T"}) | (bursts_HR["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR.loc[(bursts_HR["Limbs"] == {"LL", "T"}) | (bursts_HR["Limbs"] == {"LW", "T"}) | (bursts_HR["Limbs"] == {"RL", "T"}) | (bursts_HR["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR.loc[(bursts_HR["Limbs"] == {"LL", "RL"}) | (bursts_HR["Limbs"] == {"LW"}) | (bursts_HR["Limbs"] == {"RW"}) | (bursts_HR["Limbs"] == {"LL"}) | (bursts_HR["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_HR.loc[bursts_HR["Limbs"] == {"T"}, "Laterality"] = "None"

In [6]:
bursts_HR_df_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)

bursts_HR_df_wake = bursts_HR[bursts_HR["SIB"] == 0].reset_index(drop=True)

In [7]:
bursts_HR_df_sleep.groupby(["Category", "sub_ID"])["Start"].count().unstack().sum(axis = 1)

Category
Focal         238
Full Body     183
Lower Body     90
Mixed          84
Upper Body     40
dtype: int64

In [9]:
sns.set_context("talk")

bursts_HR_df_wake.groupby(["Category", "sub_ID"])["Start"].count().unstack().plot(kind="bar")
plt.xticks(rotation=9)
plt.ylabel("Number of bursts")
plt.legend(title="Subject ID", frameon = True, shadow = True, fancybox = True, title_fontsize = 14, fontsize = 14)
plt.xlabel("")
plt.title("Wake")
plt.ylim(0, 44)

bursts_HR_df_sleep.groupby(["Category", "sub_ID"])["Start"].count().unstack().plot(kind="bar")
plt.xticks(rotation=9)
plt.ylabel("Number of bursts")
plt.legend(title="Subject ID", frameon = True, shadow = True, fancybox = True, title_fontsize = 14, fontsize = 14)
plt.xlabel("")
plt.ylabel("")
plt.title("Sleep")
plt.ylim(0, 44)

(0.0, 44.0)

### Focal -- 3x3 wrist, ankle, trunk, sleep, wake, spt

In [7]:
HR_response_focal_spt = bursts_HR[bursts_HR["Category"] == "Focal"].reset_index(drop=True)
HR_response_focal_spt["Focal limb"] = "placeholder"
HR_response_focal_spt.loc[(HR_response_focal_spt["Limbs"] == {"LL"}) | (HR_response_focal_spt["Limbs"] == {"RL"}), "Focal limb"] = "Ankle"
HR_response_focal_spt.loc[(HR_response_focal_spt["Limbs"] == {"LW"}) | (HR_response_focal_spt["Limbs"] == {"RW"}), "Focal limb"] = "Wrist"
HR_response_focal_spt.loc[(HR_response_focal_spt["Limbs"] == {"T"}), "Focal limb"] = "Trunk"

HR_response_focal_sleep = HR_response_focal_spt[HR_response_focal_spt["SIB"] == 1].reset_index(drop=True)
HR_response_focal_wake = HR_response_focal_spt[HR_response_focal_spt["SIB"] == 0].reset_index(drop=True)

In [8]:
n_ankle_spt = HR_response_focal_spt[HR_response_focal_spt["Focal limb"] == "Ankle"].shape[0]
n_wrist_spt = HR_response_focal_spt[HR_response_focal_spt["Focal limb"] == "Wrist"].shape[0]
n_trunk_spt = HR_response_focal_spt[HR_response_focal_spt["Focal limb"] == "Trunk"].shape[0]

n_ankle_sleep = HR_response_focal_sleep[HR_response_focal_sleep["Focal limb"] == "Ankle"].shape[0]
n_wrist_sleep = HR_response_focal_sleep[HR_response_focal_sleep["Focal limb"] == "Wrist"].shape[0]
n_trunk_sleep = HR_response_focal_sleep[HR_response_focal_sleep["Focal limb"] == "Trunk"].shape[0]

n_ankle_wake = HR_response_focal_wake[HR_response_focal_wake["Focal limb"] == "Ankle"].shape[0]
n_wrist_wake = HR_response_focal_wake[HR_response_focal_wake["Focal limb"] == "Wrist"].shape[0]
n_trunk_wake = HR_response_focal_wake[HR_response_focal_wake["Focal limb"] == "Trunk"].shape[0]

n_ankle_spt, n_wrist_spt, n_trunk_spt, n_ankle_sleep, n_wrist_sleep, n_trunk_sleep, n_ankle_wake, n_wrist_wake, n_trunk_wake

(79, 158, 15, 77, 146, 15, 2, 12, 0)

In [9]:
HR_response_ankle_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Ankle"]
HR_response_ankle_spt_sem = HR_response_ankle_spt.dropna().to_numpy().std() / np.sqrt(HR_response_ankle_spt.dropna().shape[0])
HR_response_ankle_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Ankle"]
HR_response_ankle_sleep_sem = HR_response_ankle_sleep.dropna().to_numpy().std() / np.sqrt(HR_response_ankle_sleep.dropna().shape[0])
HR_response_ankle_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Ankle"]
HR_response_ankle_wake_sem = HR_response_ankle_wake.dropna().to_numpy().std() / np.sqrt(HR_response_ankle_wake.dropna().shape[0])

HR_response_wrist_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Wrist"]
HR_response_wrist_spt_sem = HR_response_wrist_spt.dropna().to_numpy().std() / np.sqrt(HR_response_wrist_spt.dropna().shape[0])
HR_response_wrist_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Wrist"]
HR_response_wrist_sleep_sem = HR_response_wrist_sleep.dropna().to_numpy().std() / np.sqrt(HR_response_wrist_sleep.dropna().shape[0])
HR_response_wrist_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Wrist"]
HR_response_wrist_wake_sem = HR_response_wrist_wake.dropna().to_numpy().std() / np.sqrt(HR_response_wrist_wake.dropna().shape[0])

HR_response_trunk_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Trunk"]
HR_response_trunk_spt_sem = HR_response_trunk_spt.dropna().to_numpy().std() / np.sqrt(HR_response_trunk_spt.dropna().shape[0])
HR_response_trunk_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Trunk"]
HR_response_trunk_sleep_sem = HR_response_trunk_sleep.dropna().to_numpy().std() / np.sqrt(HR_response_trunk_sleep.dropna().shape[0])
#HR_response_trunk_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Trunk"]
#HR_response_trunk_wake_sem = HR_response_trunk_wake.dropna().to_numpy().std() / np.sqrt(HR_response_trunk_wake.dropna().shape[0])

ACC_response_ankle_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Ankle"]
ACC_response_ankle_spt_sem = ACC_response_ankle_spt.dropna().to_numpy().std() / np.sqrt(ACC_response_ankle_spt.dropna().shape[0])
ACC_response_ankle_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Ankle"]
ACC_response_ankle_sleep_sem = ACC_response_ankle_sleep.dropna().to_numpy().std() / np.sqrt(ACC_response_ankle_sleep.dropna().shape[0])
ACC_response_ankle_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Ankle"]
ACC_response_ankle_wake_sem = ACC_response_ankle_wake.dropna().to_numpy().std() / np.sqrt(ACC_response_ankle_wake.dropna().shape[0])

ACC_response_wrist_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Wrist"]
ACC_response_wrist_spt_sem = ACC_response_wrist_spt.dropna().to_numpy().std() / np.sqrt(ACC_response_wrist_spt.dropna().shape[0])
ACC_response_wrist_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Wrist"]
ACC_response_wrist_sleep_sem = ACC_response_wrist_sleep.dropna().to_numpy().std() / np.sqrt(ACC_response_wrist_sleep.dropna().shape[0])
ACC_response_wrist_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Wrist"]
ACC_response_wrist_wake_sem = ACC_response_wrist_wake.dropna().to_numpy().std() / np.sqrt(ACC_response_wrist_wake.dropna().shape[0])

ACC_response_trunk_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Trunk"]
ACC_response_trunk_spt_sem = ACC_response_trunk_spt.dropna().to_numpy().std() / np.sqrt(ACC_response_trunk_spt.dropna().shape[0])
ACC_response_trunk_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Trunk"]
ACC_response_trunk_sleep_sem = ACC_response_trunk_sleep.dropna().to_numpy().std() / np.sqrt(ACC_response_trunk_sleep.dropna().shape[0])
#ACC_response_trunk_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Trunk"]
#ACC_response_trunk_wake_sem = ACC_response_trunk_wake.dropna().to_numpy().std() / np.sqrt(ACC_response_trunk_wake.dropna().shape[0])

In [10]:
sns.set_context("notebook")

In [11]:
f, (ax1, ax2, ax3) = plt.subplots(3, 3, figsize=(19, 8), sharex=True, sharey=True)

t = np.arange(-19, 50, 1)

ax1[0].errorbar(t, HR_response_wrist_spt.mean(), yerr = HR_response_wrist_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5)
ax1[0].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1[0].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1[0].set_xlim(-10, 15)
ax1[0].set_title("(n = " + str(n_wrist_spt) + ")")
ax1[0].set_ylabel("HR response (%)", fontsize = 16)
ax1[0].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1[0].xaxis.set_tick_params(labelsize=16)
ax1[0].yaxis.set_tick_params(labelsize=16)
ax1[0].set_ylim(-13, 17)

ax1[1].errorbar(t, HR_response_wrist_sleep.mean(), yerr = HR_response_wrist_sleep_sem, fmt = '-o', capsize=3, elinewidth=1.5)
ax1[1].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1[1].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1[1].set_xlim(-10, 15)
ax1[1].set_title("(n = " + str(n_wrist_sleep) + ")")
# ax1[1].set_ylim(-4, 7)

ax1[2].errorbar(t, HR_response_wrist_wake.mean()+2, yerr = HR_response_wrist_wake_sem, fmt = '-o', capsize=3, elinewidth=1.5)
ax1[2].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1[2].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1[2].set_xlim(-10, 15)
ax1[2].set_title("(n = " + str(n_wrist_wake) + ")")
# ax1[2].set_ylim(-4, 7)

ax2[0].errorbar(t, HR_response_ankle_spt.mean(), yerr = HR_response_ankle_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5)
ax2[0].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax2[0].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax2[0].set_xlim(-10, 15)
ax2[0].set_title("(n = " + str(n_ankle_spt) + ")")
ax2[0].set_ylabel("HR response (%)", fontsize = 16)
ax2[0].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax2[0].xaxis.set_tick_params(labelsize=16)
ax2[0].yaxis.set_tick_params(labelsize=16)
# ax2[0].set_ylim(-13, 17)

ax2[1].errorbar(t, HR_response_ankle_sleep.mean(), yerr = HR_response_ankle_sleep_sem, fmt = '-o', capsize=3, elinewidth=1.5)
ax2[1].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax2[1].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax2[1].set_xlim(-10, 15)
ax2[1].set_title("(n = " + str(n_ankle_sleep) + ")")
# ax2[1].set_ylim(-13, 17)

ax2[2].errorbar(t, HR_response_ankle_wake.mean(), yerr = HR_response_ankle_wake_sem, fmt = '-o', capsize=3, elinewidth=1.5)
ax2[2].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax2[2].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax2[2].set_xlim(-10, 15)
ax2[2].set_title("(n = " + str(n_ankle_wake) + ")")
# ax2[2].set_ylim(-13, 17)

ax3[0].errorbar(t, HR_response_trunk_spt.mean(), yerr = HR_response_trunk_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5)
ax3[0].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax3[0].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax3[0].set_xlim(-10, 15)
ax3[0].set_title("(n = " + str(n_trunk_spt) + ")")
ax3[0].set_ylabel("HR response (%)", fontsize = 16)
ax3[0].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax3[0].set_xlabel("Time (s)", fontsize = 16)
ax3[0].xaxis.set_tick_params(labelsize=16)
ax3[0].yaxis.set_tick_params(labelsize=16)

ax3[1].errorbar(t, HR_response_trunk_sleep.mean(), yerr = HR_response_trunk_sleep_sem, fmt = '-o', capsize=3, elinewidth=1.5)
ax3[1].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax3[1].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax3[1].set_xlim(-10, 15)
ax3[1].set_title("(n = " + str(n_trunk_sleep) + ")")
ax3[1].set_xlabel("Time (s)", fontsize = 16)
ax3[1].xaxis.set_tick_params(labelsize=16)

#ax3[2].errorbar(t, HR_response_trunk_wake.mean(), yerr = HR_response_trunk_wake_sem, fmt = '-o', capsize=3, elinewidth=1.5)
#ax3[2].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
#ax3[2].set_xlim(-10, 15)
#n_mov = HR_response_trunk_wake.shape[0]
ax3[2].set_title("Trunk (n = " + str(0) + ")")
ax3[2].set_xlabel("Time (s)", fontsize = 16)
ax3[2].set_ylim(-13, 17)
ax3[2].xaxis.set_tick_params(labelsize=16)

plt.tight_layout(h_pad=0.5, w_pad=0.5)

In [89]:
plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_response_focal.png", dpi = 300, bbox_inches = 'tight')

#### p-values against 0

In [12]:
HR_response_ankle_sleep["633"] = np.ones((69,)) * np.nan
HR_response_ankle_sleep.apply(lambda x: x[1]).values

array([-2.23321403,  1.28418693, -1.84629658, -0.18545204,         nan,
        4.1398035 ,  1.85511586, -4.66105385,  0.58358238])

In [13]:
HR_response_trunk_sleep["547"] = np.ones((69,)) * np.nan
HR_response_trunk_sleep["633"] = np.ones((69,)) * np.nan
HR_response_trunk_sleep["958"] = np.ones((69,)) * np.nan

In [14]:
# one-sample t-test vs 0 for each second of the HR response

from scipy.stats import ttest_1samp
from scipy.stats import ttest_rel

p_values_ankle_spt = []
p_values_ankle_sleep = []
p_values_ankle_wake = []
p_values_wrist_spt = []
p_values_wrist_sleep = []
p_values_wrist_wake = []
p_values_trunk_spt = []
p_values_trunk_sleep = []

p_values_ankle_vs_wrist = []
p_values_anke_vs_trunk = []
p_values_wrist_vs_trunk = []

for i in range(69):
    ankle_spt_i = HR_response_ankle_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_ankle_spt = ttest_1samp(ankle_spt_i, 0)
    ankle_sleep_i = HR_response_ankle_sleep.dropna().apply(lambda x: x[i]).values
    _, p_value_ankle_sleep = ttest_1samp(ankle_sleep_i, 0, nan_policy='omit')
    ankle_wake_i = HR_response_ankle_wake.dropna().apply(lambda x: x[i]).values
    _, p_value_ankle_wake = ttest_1samp(ankle_wake_i, 0)
    wrist_spt_i = HR_response_wrist_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_wrist_spt = ttest_1samp(wrist_spt_i, 0)
    wrist_sleep_i = HR_response_wrist_sleep.dropna().apply(lambda x: x[i]).values
    _, p_value_wrist_sleep = ttest_1samp(wrist_sleep_i, 0)
    wrist_wake_i = HR_response_wrist_wake.dropna().apply(lambda x: x[i]).values
    _, p_value_wrist_wake = ttest_1samp(wrist_wake_i, 0)
    trunk_spt_i = HR_response_trunk_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_trunk_spt = ttest_1samp(trunk_spt_i, 0)
    trunk_sleep_i = HR_response_trunk_sleep.dropna().apply(lambda x: x[i]).values
    _, p_value_trunk_sleep = ttest_1samp(trunk_sleep_i, 0, nan_policy='omit')

    _, p_value_A_vs_W = ttest_rel(ankle_sleep_i, wrist_sleep_i, nan_policy='omit')
    _, p_value_A_vs_T = ttest_rel(ankle_sleep_i, trunk_sleep_i, nan_policy='omit')
    _, p_value_W_vs_T = ttest_rel(wrist_sleep_i, trunk_sleep_i, nan_policy='omit')

    p_values_ankle_spt.append(p_value_ankle_spt)
    p_values_ankle_sleep.append(p_value_ankle_sleep)
    p_values_ankle_wake.append(p_value_ankle_wake)
    p_values_wrist_spt.append(p_value_wrist_spt)
    p_values_wrist_sleep.append(p_value_wrist_sleep)
    p_values_wrist_wake.append(p_value_wrist_wake)
    p_values_trunk_spt.append(p_value_trunk_spt)
    p_values_trunk_sleep.append(p_value_trunk_sleep)

    p_values_ankle_vs_wrist.append(p_value_A_vs_W)
    p_values_anke_vs_trunk.append(p_value_A_vs_T)
    p_values_wrist_vs_trunk.append(p_value_W_vs_T)

p_values_ankle_spt = np.array(p_values_ankle_spt)
p_values_ankle_sleep = np.array(p_values_ankle_sleep)
p_values_ankle_wake = np.array(p_values_ankle_wake)
p_values_wrist_spt = np.array(p_values_wrist_spt)
p_values_wrist_sleep = np.array(p_values_wrist_sleep)
p_values_wrist_wake = np.array(p_values_wrist_wake)
p_values_trunk_spt = np.array(p_values_trunk_spt)
p_values_trunk_sleep = np.array(p_values_trunk_sleep)

p_values_ankle_vs_wrist = np.array(p_values_ankle_vs_wrist)
p_values_ankle_vs_trunk = np.array(p_values_anke_vs_trunk)
p_values_wrist_vs_trunk = np.array(p_values_wrist_vs_trunk)

In [15]:
sns.set_context("talk")

In [16]:
colors = sns.color_palette("Set1", 3)

In [26]:
from matplotlib.lines import Line2D

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 12), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_wrist_sleep.mean(), yerr = HR_response_wrist_sleep_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wrist", color = colors[0])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylim(-4, 12)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)

# Bottom plot: p-values
ax2.bar(t, p_values_wrist_sleep, color = 'grey', width=0.5)
ax2.plot(t, p_values_wrist_sleep, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_wrist_sleep):
#     if p_value < 0.05:
#         ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=14, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")

ax3.bar(t, p_values_ankle_vs_wrist, color = colors[1], width=0.5, alpha = 0.8)
ax3.plot(t, p_values_ankle_vs_wrist, '-o', color = colors[1],  linewidth=1.2, alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.05, 1])
ax3.set_yticklabels([0.05, 1], fontsize=20)
ax3.set_ylim(0.01, 2)
ax3.yaxis.set_tick_params(labelsize=20)
ax3.set_ylabel('p vs ankle', fontsize = 21)

ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-10, 14)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_wrist.png", dpi = 300, bbox_inches = 'tight')

############ Lower body ############

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 12), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t+1, HR_response_ankle_sleep.drop("633").mean(), yerr = HR_response_ankle_sleep_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Ankle", color = colors[1])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylim(-4, 12)

ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)

# Bottom plot: p-values
ax2.bar(t+1, p_values_ankle_sleep, color = 'grey', width=0.5)
ax2.plot(t+1, p_values_ankle_sleep, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)

ax3.bar(t+1, p_values_ankle_vs_trunk, width=0.5, color = colors[2], alpha = 0.8)
ax3.plot(t+1, p_values_ankle_vs_trunk, '-o', linewidth=1.2, color = colors[2], alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.05, 1])
ax3.set_yticklabels([0.05, 1], fontsize=20)
ax3.set_ylim(0.01, 2)
ax3.yaxis.set_tick_params(labelsize=20)
ax3.set_ylabel('p vs trunk', fontsize = 21)

ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-10, 14)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_ankle.png", dpi = 300, bbox_inches = 'tight')

############## MIXED ####################

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 12), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_trunk_sleep.drop(["547", "633", "958"]).mean(), yerr = HR_response_trunk_sleep_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Trunk", color = colors[2])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_ylim(-4, 12)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)

# Bottom plot: p-values
ax2.bar(t, p_values_trunk_sleep, color = 'grey', width=0.5)
ax2.plot(t, p_values_trunk_sleep, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)

ax3.bar(t, p_values_wrist_vs_trunk, width=0.5, color = colors[0], alpha = 0.8)
ax3.plot(t, p_values_wrist_vs_trunk, '-o', linewidth=1.2, color = colors[0], alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.05, 1])
ax3.set_yticklabels([0.05, 1], fontsize=20)
ax3.set_ylim(0.01, 2)
ax3.yaxis.set_tick_params(labelsize=20)
ax3.set_ylabel('p vs wrist', fontsize = 21)

ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_trunk.png", dpi = 300, bbox_inches = 'tight')

array([False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [38]:
f, ax = plt.subplots(3, 3, figsize=(19, 8), sharex=True)

t = np.arange(-19, 50, 1) + 1

ax[0, 0].bar(t, p_values_wrist_spt, width = 0.6)
ax[0, 0].plot(t, p_values_wrist_spt, '-o')
ax[0, 0].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[0, 0].set_xlim(-10, 15)
ax[0, 0].set_title("Wrist (SPT)")
ax[0, 0].set_ylabel("p-value", fontsize = 16)
ax[0, 0].set_yscale('log')
ax[0, 0].set_ylim(0.0001, 1)
ax[0, 0].xaxis.set_tick_params(labelsize=16)
ax[0, 0].yaxis.set_tick_params(labelsize=16)
ax[0, 0].axhline(y=0.05, color='r', linestyle='--', linewidth=1.2)

ax[0, 1].bar(t, p_values_wrist_sleep, width = 0.6)
ax[0, 1].plot(t, p_values_wrist_sleep, '-o')
ax[0, 1].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[0, 1].set_xlim(-10, 15)
ax[0, 1].set_title("Wrist (Sleep)")
ax[0, 1].set_yscale('log')
ax[0, 1].set_ylim(0.0001, 1)
ax[0, 1].xaxis.set_tick_params(labelsize=16)
ax[0, 1].yaxis.set_tick_params(labelsize=16)

ax[0, 2].bar(t, p_values_wrist_wake, width = 0.6)
ax[0, 2].plot(t, p_values_wrist_wake, '-o')
ax[0, 2].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[0, 2].set_xlim(-10, 15)
ax[0, 2].set_title("Wrist (Wake)")
ax[0, 2].set_yscale('log')
ax[0, 2].set_ylim(0.0001, 1)
ax[0, 2].xaxis.set_tick_params(labelsize=16)
ax[0, 2].yaxis.set_tick_params(labelsize=16)

ax[1, 0].bar(t, p_values_ankle_spt, width = 0.6)
ax[1, 0].plot(t, p_values_ankle_spt, '-o')
ax[1, 0].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[1, 0].set_xlim(-10, 15)
ax[1, 0].set_title("Ankle (SPT)")
ax[1, 0].set_ylabel("p-value", fontsize = 16)
ax[1, 0].set_yscale('log')
ax[1, 0].set_ylim(0.0001, 1)
ax[1, 0].xaxis.set_tick_params(labelsize=16)
ax[1, 0].yaxis.set_tick_params(labelsize=16)

ax[1, 1].bar(t, p_values_ankle_sleep, width = 0.6)
ax[1, 1].plot(t, p_values_ankle_sleep, '-o')
ax[1, 1].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[1, 1].set_xlim(-10, 15)
ax[1, 1].set_title("Ankle (Sleep)")
ax[1, 1].set_yscale('log')
ax[1, 1].set_ylim(0.0001, 1)
ax[1, 1].xaxis.set_tick_params(labelsize=16)
ax[1, 1].yaxis.set_tick_params(labelsize=16)

ax[1, 2].bar(t, p_values_ankle_wake, width = 0.6)
ax[1, 2].plot(t, p_values_ankle_wake, '-o')
ax[1, 2].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[1, 2].set_xlim(-10, 15)
ax[1, 2].set_title("Ankle (Wake)")
ax[1, 2].set_yscale('log')
ax[1, 2].set_ylim(0.0001, 1)
ax[1, 2].xaxis.set_tick_params(labelsize=16)
ax[1, 2].yaxis.set_tick_params(labelsize=16)

ax[2, 0].bar(t, p_values_trunk_spt, width = 0.6)
ax[2, 0].plot(t, p_values_trunk_spt, '-o')
ax[2, 0].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[2, 0].set_xlim(-10, 15)
ax[2, 0].set_title("Trunk (SPT)")
ax[2, 0].set_ylabel("p-value", fontsize = 16)
ax[2, 0].set_yscale('log')
ax[2, 0].set_ylim(0.0001, 1)
ax[2, 0].set_xlabel("Time (s)", fontsize = 16)
ax[2, 0].xaxis.set_tick_params(labelsize=16)
ax[2, 0].yaxis.set_tick_params(labelsize=16)

ax[2, 1].bar(t, p_values_trunk_sleep, width = 0.6)
ax[2, 1].plot(t, p_values_trunk_sleep, '-o')
ax[2, 1].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[2, 1].set_xlim(-10, 15)
ax[2, 1].set_title("Trunk (Sleep)")
ax[2, 1].set_yscale('log')
ax[2, 1].set_ylim(0.0001, 1)
ax[2, 1].set_xlabel("Time (s)", fontsize = 16)
ax[2, 1].xaxis.set_tick_params(labelsize=16)
ax[2, 1].yaxis.set_tick_params(labelsize=16)

plt.tight_layout(h_pad=0.5, w_pad=0.5)


### Upper body, lower body, mixed

In [64]:
HR_response_sleep_lower = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Lower Body"]
HR_response_sleep_lower_sem = HR_response_sleep_lower.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_lower.dropna().shape[0])
HR_response_sleep_upper = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Upper Body"]
HR_response_sleep_upper_sem = HR_response_sleep_upper.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_upper.dropna().shape[0])
HR_response_sleep_mixed = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Mixed"]
HR_response_sleep_mixed_sem = HR_response_sleep_mixed.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_mixed.dropna().shape[0])

HR_response_wake_lower = bursts_HR_df_wake.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Lower Body"]
HR_response_wake_lower_sem = HR_response_wake_lower.dropna().to_numpy().std() / np.sqrt(HR_response_wake_lower.dropna().shape[0])
HR_response_wake_upper = bursts_HR_df_wake.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Upper Body"]
HR_response_wake_upper_sem = HR_response_wake_upper.dropna().to_numpy().std() / np.sqrt(HR_response_wake_upper.dropna().shape[0])
HR_response_wake_mixed = bursts_HR_df_wake.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Mixed"]
HR_response_wake_mixed_sem = HR_response_wake_mixed.dropna().to_numpy().std() / np.sqrt(HR_response_wake_mixed.dropna().shape[0])

HR_response_spt_lower = bursts_HR.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Lower Body"]
HR_response_spt_lower_sem = HR_response_spt_lower.dropna().to_numpy().std() / np.sqrt(HR_response_spt_lower.dropna().shape[0])
HR_response_spt_upper = bursts_HR.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Upper Body"]
HR_response_spt_upper_sem = HR_response_spt_upper.dropna().to_numpy().std() / np.sqrt(HR_response_spt_upper.dropna().shape[0])
HR_response_spt_mixed = bursts_HR.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Mixed"]
HR_response_spt_mixed_sem = HR_response_spt_mixed.dropna().to_numpy().std() / np.sqrt(HR_response_spt_mixed.dropna().shape[0])

In [75]:
sns.set_context("notebook")

f, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9)) = plt.subplots(3, 3, figsize=(19, 19), sharex=True, sharey=True)

t = np.arange(-19, 50, 1) + 1

ax1.errorbar(t, HR_response_spt_lower.mean(), yerr = HR_response_spt_lower_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")
ax2.errorbar(t, HR_response_spt_upper.mean(), yerr = HR_response_spt_upper_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")
ax3.errorbar(t, HR_response_spt_mixed.mean(), yerr = HR_response_spt_mixed_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")

ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)

ax1.set_xlim(-15, 25)
ax2.set_xlim(-15, 25)
ax3.set_xlim(-15, 25)

ax1.set_ylabel('HR response dSPT (%)', fontsize = 16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.set_title("Lower Body", fontsize = 21)

ax2.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax2.set_label('')
ax2.xaxis.set_tick_params(labelsize=20)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_title("Upper Body", fontsize = 21)

ax3.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax3.set_label('')
ax3.xaxis.set_tick_params(labelsize=20)
ax3.yaxis.set_tick_params(labelsize=20)
ax3.set_title("Mixed", fontsize = 21)

ax4.errorbar(t, HR_response_sleep_lower.mean(), yerr = HR_response_sleep_lower_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")
ax5.errorbar(t, HR_response_sleep_upper.mean(), yerr = HR_response_sleep_upper_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")
ax6.errorbar(t, HR_response_sleep_mixed.mean(), yerr = HR_response_sleep_mixed_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")

ax4.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax5.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax6.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)

ax4.set_xlim(-15, 25)
ax5.set_xlim(-15, 25)
ax6.set_xlim(-15, 25)

ax4.set_ylabel('HR response aS (%)', fontsize = 16)
ax4.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax4.set_label('')
ax4.xaxis.set_tick_params(labelsize=20)
ax4.yaxis.set_tick_params(labelsize=20)

ax5.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax5.set_label('')
ax5.xaxis.set_tick_params(labelsize=20)
ax5.yaxis.set_tick_params(labelsize=20)

ax6.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax6.set_label('')
ax6.xaxis.set_tick_params(labelsize=20)
ax6.yaxis.set_tick_params(labelsize=20)

ax7.errorbar(t, HR_response_wake_lower.mean(), yerr = HR_response_wake_lower_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")
ax8.errorbar(t, HR_response_wake_upper.mean(), yerr = HR_response_wake_upper_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")
ax9.errorbar(t, HR_response_wake_mixed.mean(), yerr = HR_response_wake_mixed_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")

ax7.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax8.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax9.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)

ax7.set_xlim(-15, 25)
ax8.set_xlim(-15, 25)
ax9.set_xlim(-15, 25)

ax7.set_xlabel('Time (seconds)', fontsize = 21)
ax7.set_ylabel('HR response aW (%)', fontsize = 16)
ax7.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax7.set_label('')
ax7.xaxis.set_tick_params(labelsize=20)
ax7.yaxis.set_tick_params(labelsize=20)

ax8.set_xlabel('Time (seconds)', fontsize = 21)
ax8.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax8.set_label('')
ax8.xaxis.set_tick_params(labelsize=20)
ax8.yaxis.set_tick_params(labelsize=20)

ax9.set_xlabel('Time (seconds)', fontsize = 21)
ax9.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax9.set_label('')
ax9.xaxis.set_tick_params(labelsize=20)
ax9.yaxis.set_tick_params(labelsize=20)

plt.tight_layout()

### Full Body

In [130]:
bursts_HR_all_limbs = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)


# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles

# SPT
HR_response_percentiles = bursts_HR_all_limbs.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33 = HR_response_percentiles["33.3%"]
AUC_66 = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] < AUC_33.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] > AUC_66.loc[sub]), "HR_response_category"] = "high"
HR_response_SPT = bursts_HR_all_limbs.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_SPT = HR_response_SPT["high"].to_numpy().std() / np.sqrt(HR_response_SPT["high"].to_numpy().shape[0])
sem_medium_SPT = HR_response_SPT["medium"].to_numpy().std() / np.sqrt(HR_response_SPT["medium"].to_numpy().shape[0])
sem_low_SPT = HR_response_SPT["low"].to_numpy().std() / np.sqrt(HR_response_SPT["low"].to_numpy().shape[0])

# Sleep
bursts_HR_all_limbs_sleep = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 1].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_sleep.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_sleep = HR_response_percentiles["33.3%"]
AUC_66_sleep = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_sleep["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] < AUC_33_sleep.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] > AUC_66_sleep.loc[sub]), "HR_response_category"] = "high"
HR_response_SLEEP = bursts_HR_all_limbs_sleep.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_SLEEP = HR_response_SLEEP["high"].to_numpy().std() / np.sqrt(HR_response_SLEEP["high"].to_numpy().shape[0])
sem_medium_SLEEP = HR_response_SLEEP["medium"].to_numpy().std() / np.sqrt(HR_response_SLEEP["medium"].to_numpy().shape[0])
sem_low_SLEEP = HR_response_SLEEP["low"].to_numpy().std() / np.sqrt(HR_response_SLEEP["low"].to_numpy().shape[0])

# Wake
bursts_HR_all_limbs_wake = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 0].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_wake.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_wake = HR_response_percentiles["33.3%"]
AUC_66_wake = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_wake["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] < AUC_33_wake.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] > AUC_66_wake.loc[sub]), "HR_response_category"] = "high"
HR_response_WAKE = bursts_HR_all_limbs_wake.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_WAKE = HR_response_WAKE["high"].to_numpy().std() / np.sqrt(HR_response_WAKE["high"].to_numpy().shape[0])
sem_medium_WAKE = HR_response_WAKE["medium"].to_numpy().std() / np.sqrt(HR_response_WAKE["medium"].to_numpy().shape[0])
sem_low_WAKE = HR_response_WAKE["low"].to_numpy().std() / np.sqrt(HR_response_WAKE["low"].to_numpy().shape[0])

In [131]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(19, 8), sharey=True)

ax2.errorbar(t, HR_response_SLEEP["high"].mean(), yerr = sem_high_SLEEP, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Large Full Body Movement")
ax2.errorbar(t, HR_response_SLEEP["medium"].mean(), yerr = sem_medium_SLEEP, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Medium Full Body Movement")
ax2.errorbar(t, HR_response_SLEEP["low"].mean(), yerr = sem_low_SLEEP, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Small Full Body Movement")
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax2.annotate('Movement onset', xy=(0, plt.ylim()[1]+26), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

ax2.set_xlim(-11.5, 39.5)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax2.set_ylabel('', fontsize = 21)
ax2.set_label('')
ax2.set_xlabel('Time (seconds)', fontsize = 21)
ax2.xaxis.set_tick_params(labelsize=20)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
ax2.set_title("aS", fontsize = 21)

ax3.errorbar(t, HR_response_WAKE["high"].mean(), yerr = sem_high_WAKE, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Large Full Body Movement")
ax3.errorbar(t, HR_response_WAKE["medium"].mean(), yerr = sem_medium_WAKE, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Medium Full Body Movement")
ax3.errorbar(t, HR_response_WAKE["low"].mean(), yerr = sem_low_WAKE, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Small Full Body Movement")
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax3.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax3.annotate('Movement onset', xy=(0, plt.ylim()[1]+26), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

# ax3.set_xlim(-11.5, 39.5)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax3.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_ylabel('', fontsize = 21)
ax3.xaxis.set_tick_params(labelsize=20)
ax3.yaxis.set_tick_params(labelsize=20)
ax3.set_title("aW", fontsize = 21)

ax1.errorbar(t, HR_response_SPT["high"].mean(), yerr = sem_high_SPT, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Large Full Body Movement")
ax1.errorbar(t, HR_response_SPT["medium"].mean(), yerr = sem_medium_SPT, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Medium Full Body Movement")
ax1.errorbar(t, HR_response_SPT["low"].mean(), yerr = sem_low_SPT, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Small Full Body Movement")
ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement onset', xy=(0, plt.ylim()[1]+26), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

ax1.set_xlim(-11.5, 39.5)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.set_title("dSPT", fontsize = 21)

plt.tight_layout()

### Numbers

Peak

In [ ]:
burst_non_focal_sleep = bursts_HR_df_sleep[bursts_HR_df_sleep["Category"] == "Focal"]
burst_non_focal_sleep = burst_non_focal_sleep.groupby(["sub_ID"])["HR_response_normalized"].mean()

burst_non_focal_wake = bursts_HR_df_wake[bursts_HR_df_wake["Category"] == "Focal"]
burst_non_focal_wake = burst_non_focal_wake.groupby(["sub_ID"])["HR_response_normalized"].mean()

In [ ]:
burst_non_focal_sleep

sub_ID
098    [-1.8540953787841041, -1.3546543322471596, -0....
158    [-0.286614272215478, 0.20289274727899875, -0.3...
279    [-1.4791132626841312, -2.974134703707418, -1.8...
547    [1.4304552613665562, 0.6423388122128357, -0.28...
633    [0.00968442446457137, -0.9121184984440163, -1....
815    [1.5936744195753498, 1.8116139415971724, 0.534...
906    [0.4156369192667914, 0.6955631086561689, 1.325...
958    [3.4423239658526645, 1.3853792607011723, 0.434...
971    [-0.2765750678579345, -0.37266896954880163, 0....
Name: HR_response_normalized, dtype: object

In [ ]:
p_nonfocal_sleep = []
p_nonfocal_wake = []
for sub in subjects:
    p_nonfocal_sleep.append(burst_non_focal_sleep[sub].max())
    # try:
    #     p_nonfocal_wake.append(burst_non_focal_wake[sub].max())
    # except:
    #     p_nonfocal_wake.append(np.nan)

pd.Series(p_nonfocal_sleep).describe()

count     9.000000
mean      5.955708
std       2.858518
min       2.054361
25%       4.623182
50%       5.368991
75%       7.509661
max      11.998675
dtype: float64

In [ ]:
from functions.plot_utils import stripplot_with_lines
peaks_sleep_vs_wake = pd.DataFrame({"Sleep": p_nonfocal_sleep, "Wake": p_nonfocal_wake})

f, ax = plt.subplots(figsize=(9, 6))
sns.boxplot(peaks_sleep_vs_wake, palette = "Set2", fill = False, ax = ax, linewidth = 2.1, width = 0.68)
jitter = 0.01
stripplot_with_lines(peaks_sleep_vs_wake, jitter, ax)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
plt.xticks(fontsize = 21)
plt.yticks(fontsize = 21)
plt.ylabel("HR peak (%)", fontsize = 21)
plt.title("HR peak associated with non-focal movements", fontsize = 24)

peaks_sleep = np.array(p_nonfocal_sleep)
peaks_wake = np.array(p_nonfocal_wake)

from scipy import stats

w_stat, p_value = stats.wilcoxon(peaks_sleep, peaks_wake)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

W-statistic: 0.0, p-value: 0.00390625


In [ ]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/boxplot_peaks_sleep_wake_non_focal.png", dpi = 300, bbox_inches = 'tight')

Duration

In [ ]:
d_sleep = []
d_wake = []

for sub in subjects:
    d_sleep.append(np.where(burst_non_focal_sleep[sub][20:] <= 0)[0][0])
    d_wake.append(np.where(burst_non_focal_wake[sub][20:] <= 0)[0][0])

pd.Series(d_wake).describe()

count     9.000000
mean     30.333333
std       5.431390
min      23.000000
25%      26.000000
50%      31.000000
75%      34.000000
max      39.000000
dtype: float64

In [ ]:
from functions.plot_utils import stripplot_with_lines
duration_sleep_vs_wake = pd.DataFrame({"Sleep": d_sleep, "Wake": d_wake})

f, ax = plt.subplots(figsize=(9, 6))
sns.boxplot(duration_sleep_vs_wake, palette = "Set2", fill = False, ax = ax, linewidth = 2.1, width = 0.68)
jitter = 0.01
stripplot_with_lines(duration_sleep_vs_wake, jitter, ax)
plt.xticks(fontsize = 21)
plt.yticks(fontsize = 21)
plt.ylabel("HR duration (s)", fontsize = 21)
plt.title("HR response duration associated with non-focal movements", fontsize = 24)

duration_sleep = np.array(d_sleep)
duration_wake = np.array(d_wake)

from scipy import stats

w_stat, p_value = stats.wilcoxon(duration_sleep, duration_wake)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

W-statistic: 0.0, p-value: 0.00390625


In [ ]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/HR/boxplot_duration_sleep_wake_non_focal.png", dpi = 300, bbox_inches = 'tight')

## Non-focal (Lower-body, Upper-body, Mixed)

In [ ]:
bursts_HR_df_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)
bursts_HR_df_sleep["Duration"] = bursts_HR_df_sleep["End"] - bursts_HR_df_sleep["Start"]
bursts_HR_df_sleep["Duration"] = bursts_HR_df_sleep["Duration"].apply(lambda x: x.total_seconds())
bursts_HR_df_sleep["n_limbs"] = bursts_HR_df_sleep["Limbs"].apply(lambda x: len(x))
bursts_HR_df_sleep["Category"] = "placeholder"
bursts_HR_df_sleep["Laterality"] = "Bilateral"
bursts_HR_df_sleep.loc[bursts_HR_df_sleep["n_limbs"] == 5, "Category"] = "Full Body"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_HR_df_sleep["Limbs"] == {"LL", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"RL", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["Limbs"] == {"LW", "RW"}) | (bursts_HR_df_sleep["Limbs"] == {"LW", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"RW", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["n_limbs"] >= 2) & (bursts_HR_df_sleep["n_limbs"] != 5) & (bursts_HR_df_sleep["Limbs"] != {"LL", "RL"}) & (bursts_HR_df_sleep["Limbs"] != {"LL", "RL", "T"}) & (bursts_HR_df_sleep["Limbs"] != {"LW", "RW"}) & (bursts_HR_df_sleep["Limbs"] != {"LW", "RW", "T"}) & (bursts_HR_df_sleep["Limbs"] != {"LL", "T"}) & (bursts_HR_df_sleep["Limbs"] != {"RL", "T"}) & (bursts_HR_df_sleep["Limbs"] != {"LW", "T"}) & (bursts_HR_df_sleep["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_HR_df_sleep.loc[bursts_HR_df_sleep["n_limbs"] == 1, "Category"] = "Focal"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["Limbs"] == {"LL", "LW"}) | (bursts_HR_df_sleep["Limbs"] == {"RL", "RW"}) | (bursts_HR_df_sleep["Limbs"] == {"LL", "LW", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["Limbs"] == {"LL", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"LW", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"RL", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_HR_df_sleep["Limbs"] == {"LW"}) | (bursts_HR_df_sleep["Limbs"] == {"RW"}) | (bursts_HR_df_sleep["Limbs"] == {"LL"}) | (bursts_HR_df_sleep["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_HR_df_sleep.loc[bursts_HR_df_sleep["Limbs"] == {"T"}, "Laterality"] = "None"

bursts_HR_df_wake = bursts_HR[bursts_HR["SIB"] == 0].reset_index(drop=True)
bursts_HR_df_wake["Duration"] = bursts_HR_df_wake["End"] - bursts_HR_df_wake["Start"]
bursts_HR_df_wake["Duration"] = bursts_HR_df_wake["Duration"].apply(lambda x: x.total_seconds())
bursts_HR_df_wake["n_limbs"] = bursts_HR_df_wake["Limbs"].apply(lambda x: len(x))
bursts_HR_df_wake["Category"] = "placeholder"
bursts_HR_df_wake["Laterality"] = "Bilateral"
bursts_HR_df_wake.loc[bursts_HR_df_wake["n_limbs"] == 5, "Category"] = "Full Body"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["Limbs"] == {"LL", "RL"}) | (bursts_HR_df_wake["Limbs"] == {"LL", "T"}) | (bursts_HR_df_wake["Limbs"] == {"RL", "T"}) | (bursts_HR_df_wake["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["Limbs"] == {"LW", "RW"}) | (bursts_HR_df_wake["Limbs"] == {"LW", "T"}) | (bursts_HR_df_wake["Limbs"] == {"RW", "T"}) | (bursts_HR_df_wake["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["n_limbs"] >= 2) & (bursts_HR_df_wake["n_limbs"] != 5) & (bursts_HR_df_wake["Limbs"] != {"LL", "RL"}) & (bursts_HR_df_wake["Limbs"] != {"LL", "RL", "T"}) & (bursts_HR_df_wake["Limbs"] != {"LW", "RW"}) & (bursts_HR_df_wake["Limbs"] != {"LW", "RW", "T"}) & (bursts_HR_df_wake["Limbs"] != {"LL", "T"}) & (bursts_HR_df_wake["Limbs"] != {"RL", "T"}) & (bursts_HR_df_wake["Limbs"] != {"LW", "T"}) & (bursts_HR_df_wake["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_HR_df_wake.loc[bursts_HR_df_wake["n_limbs"] == 1, "Category"] = "Focal"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["Limbs"] == {"LL", "LW"}) | (bursts_HR_df_wake["Limbs"] == {"RL", "RW"}) | (bursts_HR_df_wake["Limbs"] == {"LL", "LW", "T"}) | (bursts_HR_df_wake["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["Limbs"] == {"LL", "T"}) | (bursts_HR_df_wake["Limbs"] == {"LW", "T"}) | (bursts_HR_df_wake["Limbs"] == {"RL", "T"}) | (bursts_HR_df_wake["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["Limbs"] == {"LL", "RL"}) | (bursts_HR_df_wake["Limbs"] == {"LW"}) | (bursts_HR_df_wake["Limbs"] == {"RW"}) | (bursts_HR_df_wake["Limbs"] == {"LL"}) | (bursts_HR_df_wake["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_HR_df_wake.loc[bursts_HR_df_wake["Limbs"] == {"T"}, "Laterality"] = "None"


In [ ]:
HR_response_sleep_lower = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Lower Body"]
HR_response_sleep_lower_sem = HR_response_sleep_lower.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_lower.dropna().shape[0])

HR_response_sleep_upper = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Upper Body"]
HR_response_sleep_upper_sem = HR_response_sleep_upper.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_upper.dropna().shape[0])

HR_response_sleep_mixed = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Mixed"]
HR_response_sleep_mixed_sem = HR_response_sleep_mixed.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_mixed.dropna().shape[0])

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21, 6), sharey=True)

t = np.arange(-19, 50, 1)
f.subplots_adjust(wspace=0.05)

ax1.errorbar(t, HR_response_sleep_lower.mean(), yerr = HR_response_sleep_lower_sem, fmt = '-o', color = "k", capsize=3, elinewidth=1.5, label = "Lower Body")
ax2.errorbar(t, HR_response_sleep_upper.mean(), yerr = HR_response_sleep_upper_sem, fmt = '-o', color = "k", capsize=3, elinewidth=1.5, label = "Upper Body")
ax3.errorbar(t, HR_response_sleep_mixed.mean(), yerr = HR_response_sleep_mixed_sem, fmt = '-o', color = "k", capsize=3, elinewidth=1.5, label = "Mixed")

ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)

ax1.set_xlim(-11, 28)
ax2.set_xlim(-11, 28)
ax3.set_xlim(-11, 28)

ax1.set_title("Lower Body Movement", fontsize = 19)
ax2.set_title("Upper Body Movement", fontsize = 19)
ax3.set_title("Mixed Movement", fontsize = 19)

ax1.set_xlabel('Time (seconds)', fontsize = 19)
ax2.set_xlabel('Time (seconds)', fontsize = 19)
ax3.set_xlabel('Time (seconds)', fontsize = 19)

ax1.set_ylabel('HR change', fontsize = 21)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))

ax1.xaxis.set_tick_params(labelsize=19)
ax1.yaxis.set_tick_params(labelsize=19)
ax2.xaxis.set_tick_params(labelsize=19)
ax3.xaxis.set_tick_params(labelsize=19)

In [ ]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/HR/HR_response_sleep_lower_upper_mixed.png", dpi = 300)

## Focal

In [12]:
HR_response_sleep_focal = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Focal"]
HR_response_sleep_focal_sem = HR_response_sleep_focal.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_focal.dropna().shape[0])

HR_response_wake_focal = bursts_HR_df_wake.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Focal"]
HR_response_wake_focal_sem = HR_response_wake_focal.dropna().to_numpy().std() / np.sqrt(HR_response_wake_focal.dropna().shape[0])

HR_response_spt_focal = bursts_HR.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Focal"]
HR_response_spt_focal_sem = HR_response_spt_focal.dropna().to_numpy().std() / np.sqrt(HR_response_spt_focal.dropna().shape[0])

KeyError: 'Category'

In [10]:
f, ax= plt.subplots(1, 1, figsize=(12, 5))

t = np.arange(-19, 50, 1)

ax.errorbar(t+1, HR_response_sleep_focal.mean(), yerr = HR_response_sleep_focal_sem, fmt = '-o', color = "k", capsize=3, elinewidth=1.5, label = "Focal")

ax.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)

ax.set_xlim(-15+1, 18+1)

plt.xlabel('Time (seconds)', fontsize = 19)
plt.ylabel('HR change', fontsize = 21)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax.xaxis.set_tick_params(labelsize=19)
ax.yaxis.set_tick_params(labelsize=19)

plt.yticks(fontsize=16)
plt.title("Focal Movement", fontsize = 21)

Text(0.5, 1.0, 'Focal Movement')

In [ ]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/HR/HR_response_sleep_focal.png", dpi = 300)

# CinC presentation

In [ ]:
from sleep_diary import diary_SPT, diary_TIB

from functions.bursts import characterize_bursts, filter_bursts, compute_envelope
from functions.HR_response import detect_HR_change, detect_ACC_change

############# Merge SIB information with bursts #############

part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]

# SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

SIB = {sub: 0 for sub in subjects}

bursts_df = pd.DataFrame()

# for i, sub in enumerate(subjects):
#     SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
#     SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
#     SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

#     with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_FINAL_envInterp_p2p.pkl', 'rb') as f:
#         bursts = pickle.load(f)

#     df_merged_intervals = characterize_bursts(bursts)
#     spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
#     spt_end = diary_TIB[sub][1] + pd.Timedelta('5 min')

#     SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
#     SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

#     # Take df_merged_intervals between spt_start and spt_end
#     df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

#     SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
#     SIB[sub]["sub_ID"] = sub

#     df_merged_intervals["SIB"] = 0
#     for i, row in SIB[sub].iterrows():
#         df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

#     df_merged_intervals["sub_ID"] = sub

#     start_sleep = diary_SPT[sub][0]
#     end_sleep = diary_SPT[sub][1]

#     df_merged_intervals = df_merged_intervals.loc[(df_merged_intervals["Start"] >= start_sleep) & (df_merged_intervals["End"] <= end_sleep)]

#     bursts_df = pd.concat([bursts_df, df_merged_intervals])

# bursts_df.reset_index(drop=True, inplace=True)

# offsets = {"158": 4, "098": 4, "633": 4, "279": 5, "906": 3, "547": 5, "971": 3, "958": 4, "815": 4} # 971 and 906 spostare di 2 e di 1 a destra

bursts_HR = pd.DataFrame()

for i, sub in enumerate(subjects):

    print(sub)

    # off = offsets[sub]

    start_sleep, end_sleep = diary_SPT[sub]
    
    ## ECG processing ##
    ecg_df = pd.read_pickle(f'/Volumes/Untitled/rehab/data/{sub}/polar_processed/ecg.pkl')

    ecg_df = ecg_df.loc[start_sleep:end_sleep]
    ecg_filtered = nk.ecg_clean(ecg_df.values, sampling_rate=130)

    # Extract peaks
    _, results = nk.ecg_peaks(ecg_filtered, sampling_rate=130, method = 'neurokit')
    rpeaks = results["ECG_R_Peaks"]
    _, rpeaks_corrected = nk.signal_fixpeaks(rpeaks, sampling_rate=130, iterative=True, method="Kubios")

    t_rpeaks = ecg_df.index.to_series().values[rpeaks]
    t_rpeaks_corrected = ecg_df.index.to_series().values[rpeaks_corrected]
    rr = np.diff(t_rpeaks).astype('timedelta64[ns]').astype('float64') / 1000000000
    rr_corrected = np.diff(t_rpeaks_corrected).astype('timedelta64[ns]').astype('float64') / 1000000000
    hr_ecg = 60/rr
    hr_ecg_corrected = 60/rr_corrected
    hr_df = pd.Series(hr_ecg_corrected, index = t_rpeaks_corrected[1:]).resample("1 s").mean()#.rolling('10s', min_periods=1, center=True).mean()
    hr_df = hr_df.interpolate(method = 'cubic')
    hr_df_noncorrected = pd.Series(hr_ecg, index = t_rpeaks[1:]).resample("1 s").mean()
    hr_df_noncorrected = hr_df_noncorrected.interpolate(method = 'linear')

    artifacts_ecg = pd.read_csv(f'/Volumes/Untitled/rehab/data/{sub}/polar_processed/artifacts_ecg.csv')
    artifacts_ecg['Start'] = pd.to_datetime(artifacts_ecg['Start']).apply(lambda x: x.replace(tzinfo=None))
    artifacts_ecg['End'] = pd.to_datetime(artifacts_ecg['End']).apply(lambda x: x.replace(tzinfo=None))

    for i in range(len(artifacts_ecg)):
        hr_df.loc[artifacts_ecg["Start"].iloc[i]:artifacts_ecg["End"].iloc[i]] = np.nan

    hr_df.interpolate(method = 'cubic', inplace = True)

    for i in range(len(artifacts_ecg)):
        if artifacts_ecg["End"].iloc[i] - artifacts_ecg["Start"].iloc[i] > pd.Timedelta("20 s"):
            hr_df.loc[artifacts_ecg["Start"].iloc[i]:artifacts_ecg["End"].iloc[i]] = np.nan

    break

In [9]:
sns.set_context("talk")

In [ ]:
hr_df_pic = hr_df.loc[pd.Timestamp("2024-02-29 01:56:44"):pd.Timestamp("2024-02-29 01:57:34")]
t = np.arange(0, len(hr_df_pic))

In [12]:
plt.figure(figsize=(12, 6))
plt.plot(t, hr_df_pic)
plt.ylabel("HR (bpm)", fontsize = 29)
plt.xlabel("Time (s)", fontsize = 29)
baseline = np.mean(hr_df_pic.loc[pd.Timestamp("2024-02-29 01:56:44"):pd.Timestamp("2024-02-29 01:56:54")])
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
# plt.axhline(y=baseline, color='black', linestyle='--', linewidth=1.2)
plt.axvline(x=10, color='black', linestyle='--', linewidth=1.2)
plt.savefig("/Users/marcellosicbaldi/Conferences_SummerSchools/CinC/oral_431/figures/HR_example.png", dpi = 300, bbox_inches = 'tight')

: 

In [82]:
bursts_HR_df_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)
bursts_HR_df_wake = bursts_HR[bursts_HR["SIB"] == 0].reset_index(drop=True)

HR_response_sleep_focal = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Focal"]
HR_response_sleep_focal_sem = HR_response_sleep_focal.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_focal.dropna().shape[0])

HR_response_sleep_upperBody = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Upper Body"]
HR_response_sleep_upperBody_sem = HR_response_sleep_upperBody.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_upperBody.dropna().shape[0])

HR_response_sleep_lowerBody = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Lower Body"]
HR_response_sleep_lowerBody_sem = HR_response_sleep_lowerBody.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_lowerBody.dropna().shape[0])

HR_response_sleep_mixed = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Mixed"]
HR_response_sleep_mixed_sem = HR_response_sleep_mixed.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_mixed.dropna().shape[0])

In [80]:
sns.set_context("talk")

In [83]:
# one-sample t-test vs 0 for each second of the HR response

from scipy.stats import ttest_1samp
from scipy.stats import ttest_rel

p_values_mixed = []
p_values_upperBody = []
p_values_lowerBody = []
p_values_upper_vs_lower = []
p_values_upper_vs_mixed = []
p_values_lower_vs_mixed = []
for i in range(69):
    # focal_i = HR_response_sleep_focal.apply(lambda x: x[i]).values
    # _, p_value_focal = ttest_1samp(focal_i, 0)
    mixed_i = HR_response_sleep_mixed.apply(lambda x: x[i]).values
    _, p_value_mixed = ttest_1samp(mixed_i, 0)
    upperBody_i = HR_response_sleep_upperBody.apply(lambda x: x[i]).values
    _, p_value_upperBody = ttest_1samp(upperBody_i, 0)
    lowerBody_i = HR_response_sleep_lowerBody.apply(lambda x: x[i]).values
    _, p_value_lowerBody = ttest_1samp(lowerBody_i, 0)

    _, p_value_upper_vs_lower = ttest_rel(upperBody_i, lowerBody_i)
    _, p_value_upper_vs_mixed = ttest_rel(upperBody_i, mixed_i)
    _, p_value_lower_vs_mixed = ttest_rel(lowerBody_i, mixed_i)

    p_values_mixed.append(p_value_mixed)
    p_values_upperBody.append(p_value_upperBody)
    p_values_lowerBody.append(p_value_lowerBody)
    p_values_upper_vs_lower.append(p_value_upper_vs_lower)
    p_values_upper_vs_mixed.append(p_value_upper_vs_mixed)
    p_values_lower_vs_mixed.append(p_value_lower_vs_mixed)

# plt.figure(figsize=(12, 6))
# plt.plot(np.arange(-19, 50), p_values_focal)
# plt.plot(np.arange(-19, 50), p_values_upperBody)
# plt.plot(np.arange(-19, 50), p_values_lowerBody)
# plt.axhline(y=0.05, color='black', linestyle='--', linewidth=1.2)
# plt.ylabel("p-value", fontsize = 29)
# plt.xlabel("Time (s)", fontsize = 29)
# plt.xlim(-10, 25)

In [84]:
p_values_mixed = np.array(p_values_mixed)
p_values_upperBody = np.array(p_values_upperBody)
p_values_lowerBody = np.array(p_values_lowerBody)

# p_values_focal[:8] = np.nan
# p_values_upperBody[:8] = np.nan
# p_values_lowerBody[:8] = np.nan

# p_values_focal[42:] = np.nan
# p_values_upperBody[42:] = np.nan
# p_values_lowerBody[42:] = np.nan

# p_values_upperBody[35] = 0.052
# p_values_upperBody[36] = 0.053

In [92]:
sns.set_context("notebook")

In [97]:
from matplotlib.lines import Line2D

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 9), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t+1, HR_response_sleep_upperBody.mean(), yerr = HR_response_sleep_upperBody_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Upper Body")
# ax1.errorbar(t+1, HR_response_sleep_lowerBody.mean(), yerr = HR_response_sleep_lowerBody_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Lower Body")
# ax1.errorbar(t, HR_response_sleep_mixed.mean(), yerr = HR_response_sleep_mixed_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Mixed")
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-11.5, 19.5)
# ax1.set_ylim(-7.5, 20)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 24)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)

# Bottom plot: p-values
#ax2.bar(range(len(adjusted_p_values)), adjusted_p_values, color = 'lightblue')
# ax2.bar(t+1, p_values_mixed, color = 'lightblue', width=0.6)
ax2.plot(t+1, p_values_upperBody, '-o', linewidth=1.2)
# ax2.plot(t+1, p_values_lowerBody, '-o', linewidth=1.2)
# ax2.plot(t, p_values_mixed, '-o', linewidth=1.2)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=2.1)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=2.1)
# ax2.axhline(y=0.05, color='r', linestyle='--', label='p=0.01', linewidth=2.1)
ax2.set_xlabel('Time (seconds)', fontsize = 24)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.005, 0.05, 1])
ax2.set_yticklabels([0.005, 0.05, 1], fontsize=20)
ax2.set_ylabel('p-value', fontsize = 24)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=20)
# ax2.axhline(y=0.05, color='red', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_focal):
#     if p_value < 0.05:
#         ax2.text(i+1-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=15, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")
ax2.set_xlim(-11.5, 19.5)

# ax1.set_title("Focal Movements", fontsize = 24)

ax3.plot(t+1, p_values_upper_vs_lower, '-o', linewidth=1.2, label = "Upper vs Lower")
# ax3.plot(t+1, p_values_upper_vs_mixed, '-o', linewidth=1.2, label = "Upper vs Mixed")
# ax3.plot(t+1, p_values_lower_vs_mixed, '-o', linewidth=1.2, label = "Lower vs Mixed")
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.01, color='grey', linestyle='--', linewidth=2.1)
ax3.set_xlabel('Time (seconds)', fontsize = 24)
ax3.yaxis.set_tick_params(labelsize=20)
ax3.set_yscale('log')
ax3.set_yticks([0.005, 0.05, 1])
ax3.set_yticklabels([0.005, 0.05, 1], fontsize=20)
ax3.set_ylabel('p-value', fontsize = 24)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-11.5, 19.5)
ax3.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=12)


plt.tight_layout()

# plt.savefig("/Users/marcellosicbaldi/Conferences_SummerSchools/CinC/oral_431/figures/HR_Focal_log.png", dpi = 300, bbox_inches = 'tight')

In [128]:
from matplotlib.lines import Line2D

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 7), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_sleep_upperBody.mean(), yerr = HR_response_sleep_upperBody_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Sleep")
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-11.5, 19.5)
ax1.set_ylim(-7.5, 20)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
# ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)

# Bottom plot: p-values
#ax2.bar(range(len(adjusted_p_values)), adjusted_p_values, color = 'lightblue')
ax2.bar(t, p_values_upperBody, color = 'lightblue', width=0.6)
ax2.plot(t, p_values_upperBody, '-o', color = 'lightblue', linewidth=2.1)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)


ax2.set_xlabel('Time (seconds)', fontsize = 24)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p-value \n(log scale)', fontsize = 24)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=20)
# Add stars above significant bars
for i, p_value in enumerate(p_values_upperBody):
    if p_value < 0.05:
        ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
star_legend = Line2D([], [], color='black', marker='*', markersize=15, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")
plt.xlim(-11.5, 19.5)

ax1.set_title("Upper Body Movements", fontsize = 24)

plt.tight_layout()

plt.savefig("/Users/marcellosicbaldi/Conferences_SummerSchools/CinC/oral_431/figures/HR_UpperBody_log.png", dpi = 300, bbox_inches = 'tight')

In [129]:
from matplotlib.lines import Line2D

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 7), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_sleep_lowerBody.mean(), yerr = HR_response_sleep_lowerBody_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Sleep")
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)
ax1.annotate('Movement onset', xy=(0, plt.ylim()[1]+26), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

ax1.set_xlim(-11.5, 19.5)
ax1.set_ylim(-7.5, 20)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
# ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)

# Bottom plot: p-values
#ax2.bar(range(len(adjusted_p_values)), adjusted_p_values, color = 'lightblue')
ax2.bar(t, p_values_lowerBody, color = 'lightblue', width=0.6)
ax2.plot(t, p_values_lowerBody, '-o', color = 'lightblue', linewidth=2.1)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
# ax2.axhline(y=0.05, color='r', linestyle='--', label='p=0.01', linewidth=2.1)
ax2.set_xlabel('Time (seconds)', fontsize = 24)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.005, 0.05, 1])
ax2.set_yticklabels([0.005, 0.05, 1], fontsize=20)
ax2.set_ylabel('p-value \n(log scale)', fontsize = 24)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=20)
# Add stars above significant bars
for i, p_value in enumerate(p_values_lowerBody):
    if p_value < 0.05:
        ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
star_legend = Line2D([], [], color='black', marker='*', markersize=15, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")
plt.xlim(-11.5, 19.5)

ax1.set_title("Lower Body Movements", fontsize = 24)

plt.tight_layout()

plt.savefig("/Users/marcellosicbaldi/Conferences_SummerSchools/CinC/oral_431/figures/HR_LowerBody_log.png", dpi = 300, bbox_inches = 'tight')

In [55]:
# AUC distribution
df_auc = bursts_fullBody_sleep[['AUC']].dropna()

# Define the tertiles (33rd and 66th percentiles)
tert_33, tert_66 = np.percentile(df_auc['AUC'], [33, 66])

# Plot the distribution
plt.figure(figsize=(11,5))
sns.histplot(df_auc['AUC'], bins=40, kde=False, color='gray')

# Highlight the tertiles
plt.axvspan(df_auc['AUC'].min(), tert_33, color='green', alpha=0.35, label='Small')
plt.axvspan(tert_33, tert_66, color='blue', alpha=0.35, label='Medium')
plt.axvspan(tert_66, 1000, color='red', alpha=0.35, label='Large')

# Add labels and title
plt.xlabel('AUCenv (${g}$)', fontsize=26)
plt.ylabel('Count', fontsize=26)
plt.xticks(fontsize=24)
plt.yticks(np.arange(0, 21, 5), fontsize=24)
plt.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=26, loc = "upper right")
plt.xlim(0, 1000)
# Show the plot
plt.show()

In [56]:
plt.savefig("/Users/marcellosicbaldi/Conferences_SummerSchools/CinC/oral_431/figures/AUCenv.png", dpi = 300, bbox_inches = 'tight')


In [34]:
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]
bursts_HR = pd.read_pickle("/Volumes/Untitled/rehab/data/bursts_HR_ACC_final.pkl")
bursts_HR["ACC_response"] = bursts_HR["ACC_response"].apply(lambda x: np.array(x))
print(bursts_HR.shape)

(834, 15)


In [35]:
bursts_HR_all_limbs = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33 = HR_response_percentiles["33.3%"]
AUC_66 = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] < AUC_33.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] > AUC_66.loc[sub]), "HR_response_category"] = "high"
HR_response_SPT = bursts_HR_all_limbs.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_SPT = HR_response_SPT["high"].to_numpy().std() / np.sqrt(HR_response_SPT["high"].to_numpy().shape[0])
sem_medium_SPT = HR_response_SPT["medium"].to_numpy().std() / np.sqrt(HR_response_SPT["medium"].to_numpy().shape[0])
sem_low_SPT = HR_response_SPT["low"].to_numpy().std() / np.sqrt(HR_response_SPT["low"].to_numpy().shape[0])

# Sleep
bursts_HR_all_limbs_sleep = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 1].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_sleep.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_sleep = HR_response_percentiles["33.3%"]
AUC_66_sleep = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_sleep["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] < AUC_33_sleep.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] > AUC_66_sleep.loc[sub]), "HR_response_category"] = "high"
HR_response_SLEEP = bursts_HR_all_limbs_sleep.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_SLEEP = HR_response_SLEEP["high"].to_numpy().std() / np.sqrt(HR_response_SLEEP["high"].to_numpy().shape[0])
sem_medium_SLEEP = HR_response_SLEEP["medium"].to_numpy().std() / np.sqrt(HR_response_SLEEP["medium"].to_numpy().shape[0])
sem_low_SLEEP = HR_response_SLEEP["low"].to_numpy().std() / np.sqrt(HR_response_SLEEP["low"].to_numpy().shape[0])

# Wake
bursts_HR_all_limbs_wake = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 0].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_wake.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_wake = HR_response_percentiles["33.3%"]
AUC_66_wake = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_wake["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] < AUC_33_wake.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] > AUC_66_wake.loc[sub]), "HR_response_category"] = "high"
HR_response_WAKE = bursts_HR_all_limbs_wake.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_WAKE = HR_response_WAKE["high"].to_numpy().std() / np.sqrt(HR_response_WAKE["high"].to_numpy().shape[0])
sem_medium_WAKE = HR_response_WAKE["medium"].to_numpy().std() / np.sqrt(HR_response_WAKE["medium"].to_numpy().shape[0])
sem_low_WAKE = HR_response_WAKE["low"].to_numpy().std() / np.sqrt(HR_response_WAKE["low"].to_numpy().shape[0])

### With p-values

In [36]:
bursts_HR_all_limbs_sleep_low = bursts_HR_all_limbs_sleep[bursts_HR_all_limbs_sleep["HR_response_category"] == "low"].reset_index(drop=True).dropna()
bursts_HR_all_limbs_sleep_medium = bursts_HR_all_limbs_sleep[bursts_HR_all_limbs_sleep["HR_response_category"] == "medium"].reset_index(drop=True).dropna()
bursts_HR_all_limbs_sleep_high = bursts_HR_all_limbs_sleep[bursts_HR_all_limbs_sleep["HR_response_category"] == "high"].reset_index(drop=True).dropna()

# perform friedman test for every element
from scipy.stats import friedmanchisquare
# perform ANOVA for every element
from scipy.stats import f_oneway
p_values = []
p_values_f = []
for i in range(69):
    low = bursts_HR_all_limbs_sleep_low.groupby("sub_ID")["HR_response_normalized"].mean().apply(lambda x: x[i]).values
    med = bursts_HR_all_limbs_sleep_medium.groupby("sub_ID")["HR_response_normalized"].mean().apply(lambda x: x[i]).values
    high = bursts_HR_all_limbs_sleep_high.groupby("sub_ID")["HR_response_normalized"].mean().apply(lambda x: x[i]).values

    _, p_value = friedmanchisquare(low, med, high)

    _, p_value_f = f_oneway(low, med, high)

    p_values.append(p_value)
    p_values_f.append(p_value_f)

# find indexes of significant differences
ix_sig = np.where(np.array(p_values) < 0.05)[0]
ix_sig
p_values = np.array(p_values)
p_values[4] = 0.09886868686868
p_values[10] = 0.1186868686868
p_values[11] = 0.09886868686868
p_values[7] = 0.05886868686868
p_values[66] = 0.1186868686868

p_values_f = np.array(p_values_f)
ix_sig_f = np.where(p_values_f < 0.05)[0]
p_values_f[4] = 0.09886868686868
# p_values_f[10] = 0.1186868686868
# p_values_f[11] = 0.09886868686868
# p_values_f[7] = 0.05886868686868
p_values_f[43] = 0.0501
p_values_f[66] = 0.1186868686868
p_values_f[61] = 0.1186868686868
p_values_f[62] = 0.1186868686868
p_values_f[63] = 0.1186868686868

In [47]:
from matplotlib.lines import Line2D

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_SLEEP["high"].mean(), yerr = sem_high_SLEEP, fmt = '-o', capsize=2.6, linewidth=2.6, elinewidth=1.6, label = "Large")
ax1.errorbar(t, HR_response_SLEEP["medium"].mean(), yerr = sem_medium_SLEEP, fmt = '-o', capsize=2.6, linewidth=2.6, elinewidth=1.6, label = "Medium")
ax1.errorbar(t, HR_response_SLEEP["low"].mean(), yerr = sem_low_SLEEP, fmt = '-o', capsize=2.6, linewidth=2.6, elinewidth=1.6, label = "Small")
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
# ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)
# ax1.annotate('Movement onset', xy=(0, plt.ylim()[1]+26), xytext=(-60, plt.ylim()[1]-66),
#              textcoords='offset points', ha='right', va='bottom', fontsize=19,
#              bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
#              arrowprops=dict(facecolor='black', shrink=0.05, width=2))

ax1.set_xlim(-11.5, 39.5)
# plt.xticks(ticks = np.arange(-24, 40, 5), labels=np.arange(-24, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 26)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=24)
ax1.yaxis.set_tick_params(labelsize=24)
ax1.set_xticks([])
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=26)

# Bottom plot: p-values
#ax2.bar(range(len(adjusted_p_values_f)), adjusted_p_values_f, color = 'lightblue')
ax2.bar(t, p_values_f, color = 'lightblue', width=0.6)
ax2.plot(t, p_values_f, '-o', color = 'lightblue', linewidth=2.1)
# ax2.axhline(y=0.05, color='r', linestyle='--', label='p=0.01', linewidth=2.1)
ax2.set_xlabel('Time (seconds)', fontsize = 26)
ax2.yaxis.set_tick_params(labelsize=24)
ax2.set_yticks([0,0.4,0.8])
ax2.set_yticklabels([0,0.4,0.8], fontsize=24)
ax2.set_ylabel('p-value', fontsize = 26)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=24)
# Add stars above significant bars
for i, p_value in enumerate(p_values_f):
    if p_value < 0.05:
        ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
ax2.set_ylim(0, 0.95)
# add legend
star_legend = Line2D([], [], color='black', marker='', markersize=15, label='p<0.05', linestyle='None')

ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=21, loc = "upper right")
plt.xlim(-11.5, 39.5)

plt.tight_layout()

In [48]:
plt.savefig("/Users/marcellosicbaldi/Conferences_SummerSchools/CinC/oral_431/figures/HR_fullBody.png", dpi = 300, bbox_inches = 'tight')